# "A new home"
> "search in progress"

- toc: false
- badges: true
- comments: false
- categories: [altair]

In [1]:
#hide
#- image: images/copied_from_nb/2021-05-01-our-slice-of-paradise.png
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import requests

import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

In [3]:
locations = {
    "Karlsruhe": (49.0094, 8.4044, 118),
    "Tenerife":  (28.4682, -16.2546, 55),
    "Lisbon":    (38.7167, -9.1333, 45),
    "Mallorca":  (39.55, 2.7333, 4)
}

climate = None

base_url = "https://api.meteostat.net/v2/point/climate"
headers = {
    "x-api-key": os.getenv("meteostat_api_key")
}

df = None

for location, coords in locations.items():
    
    lat, lon, elev = coords

    params = {
        "lat": lat,
        "lon": lon,
        "alt": elev
    }

    response = requests.get(base_url, params=params, headers=headers)
    
    _df = pd.DataFrame(response.json()["data"])
    _df["location"] = location
    
    if df is None:
        df = _df
    else:
        df = df.append(_df)

In [4]:
base_chart = alt.Chart(df)

label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

alt.layer(
    base_chart.encode(
        x="month:O",
        y="tmax",
        y2="tmin",
        color="location"
    ).mark_area(fillOpacity=.25) ,

    base_chart.encode(
        x="month:O",
        y="tavg",
        color="location"
    ).mark_line(),

        # add a rule mark to serve as a guide line
    alt.Chart().mark_rule(color='#aaa').encode(
            x='month:O'
        ).transform_filter(label),

    # add circle marks for selected time points, hide unselected points
    base_chart.mark_circle().encode(
            opacity=alt.condition(label, alt.value(1), alt.value(0))
        ).add_selection(label)
).properties(
    width=800,
    height=400
)

alt.LayerChart(...)